## Query Processing Module

#### Query Input

In [1]:
import nltk
# nltk.download('stopwords')

DATA="../data/"

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
print(tf.__version__)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

1.15.0


In [3]:
from nltk.metrics import edit_distance
#Definitions dictionary
import os, json
import pickle
defdict = {}
path_to_json = DATA + 'Definitions/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
for i in json_files:
    with open(DATA + 'Definitions/'+i) as json_file:
        data = json.load(json_file)
    defdict.update(data)
definitions = list(defdict.keys())
defvalues = list(defdict.values())

docs = ["Issue and Listing of Non-Convertible Redeemable Preference Shares", "Investment Advisers", "Depositories and Participants", "Mutual Funds", "Employees' Service", "Substantial Acquisition of Shares and Takeovers", "Appointment of Administrator and Procedure for Refunding to the Investors", "Prohibition of Fraudulent and Unfair Trade Practices relating to Securities Market", "Know Your Client", "Prohibition of Insider Trading", "Merchant Bankers", "Issue and Listing of Securitised Debt Instruments and Security Receipts", "Delisting of Equity Shares", "Issue of Capital and Disclosure Requirements", "Foreign Venture Capital Investor", "Procedure for Board Meetings", "Custodian", "Ombudsman", "Investor Protection and Education Fund", "Foreign Portfolio Investors", "Issue of Sweat Equity", "Collective Investment Scheme", "Portfolio Managers", "Research Analysts", "Procedure for Search and Seizure", "Issue Of Capital And Disclosure Requirements", "Share Based Employee Benefits", "Debenture Trustees", "Alternative Investment Funds", "Stock Exchanges and Clearing Corporations", " Self Regulatory Organisations", "Settlement Proceedings", "Issue and Listing of Municipal Debt Securities", "Buy-back of Securities", "Issue and Listing of Debt Securities", "Infrastructure Investment Trusts", " Stock Brokers", "Listing Obligations and Disclosure Requirements", "Registrars to an Issue and Share Transfer Agents", "Real Estate Investment Trusts", "Intermediaries", "Certification of Associated Persons in the Securities Markets", "Credit Rating Agencies", "Regulatory Fee on Stock Exchanges", "Underwriters", "Buy Back Of Securities", "Bankers to an Issue", "Central Database of Market Participants"]

#Regulations
finalclean48 = []
with open(DATA + 'cleanedregulations48.pkl','rb') as f:
    finalclean48 = pickle.load(f)    
docregs = finalclean48

#Topics documentwise
finalcleantopics48 = []
with open(DATA + 'cleanedregtopics48.pkl','rb') as f:
    finalcleantopics48 = pickle.load(f)    
finaltopics = finalcleantopics48

#vocab definitions
mainvocab = []
with open(DATA + 'mainvocab.pkl','rb') as f:
    mainvocab = pickle.load(f) 

vocabdef = []
with open(DATA + 'vocabdef.pkl','rb') as f:
    vocabdef = pickle.load(f) 

#Additional Documents

with open(DATA + 'concept_filenames.pkl','rb') as f:
    cfile = pickle.load(f) 
with open(DATA + 'concept_sentences.pkl','rb') as f:
    csent = pickle.load(f) 
with open(DATA + 'concept_text.pkl','rb') as f:
    ctext = pickle.load(f) 

with open(DATA + 'informal_filenames.pkl','rb') as f:
    ifile = pickle.load(f) 
with open(DATA + 'informal_sentences.pkl','rb') as f:
    isent = pickle.load(f) 
with open(DATA + 'informal_text.pkl','rb') as f:
    itext = pickle.load(f) 
    
with open(DATA + 'miscvocab.pkl', 'rb') as f:
    miscvocab = pickle.load(f)
    
#Legal Case files
with open(DATA + 'case_filenames.pkl','rb') as f:
    lfile = pickle.load(f) 
with open(DATA + 'case_sentences.pkl','rb') as f:
    lsent = pickle.load(f) 
with open(DATA + 'case_text.pkl','rb') as f:
    ltext = pickle.load(f) 
    
with open(DATA + 'casevocab.pkl', 'rb') as f:
    casevocab = pickle.load(f)

import spacy
nlp = spacy.load('en_core_web_sm')

from nltk.corpus import wordnet

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import math
import numpy as np
from collections import Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction import stop_words
STOPWORDS = set(
    list(stop_words.ENGLISH_STOP_WORDS) +\
    stopwords.words('english') +\
    ['mm', 'section', 'subsection', 'schedule', '-PRON-', 'chapter', 'regulation', 'repealed', 'thereto','unpublishe', 'thereunder','guideline', 'reference','onus','make','Page','Securities','Exchange','India'])

from cdqa.pipeline import QAPipeline

from cdqa.utils.download import download_model
download_model(model='bert-squad_1.1', dir='./models')

with open(DATA + 'glossary.json') as f:
    glossary = json.load(f)

/home/spawnfire/anaconda3/envs/testingEnvironment/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [4]:
def queryvocab(query):
    query1 = ""
    qvocab=[]
    for i in query.split():
        if(i not in stopwords.words()):
            if(i != 'What' and i != 'what' and i!='How'):
                qvocab.append(i)
                query1 = query1 + " " + i
    query = query1
    return qvocab, query

In [5]:
def regextract(docinput, docs, docregs, mainvocab, vocabdef, query, glossary):
    
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    v = []
    
    if(docinput == 'All'):
        rdocs = []
        for i in glossary.keys():
            if(i in query.lower()):
                dval = glossary[i]
                for j in dval:
                    if(j not in rdocs):
                        rdocs.append(j)
        if(len(rdocs)>0):
            docinput = []
            for i in rdocs:
                docinput.append(docs[i])
        else:
            docinput = ['All']
            
    else:
        d = [docinput]
        docinput = d
    
    for dval in docinput:
    
        if(dval != 'All'):
            ind = docs.index(dval)

            rules = docregs[ind]
        
            t=-1
            vocab0 = []
            for i in rules:
                t=t+1
                vocab0.append(i.split())
            
            for i in vocab0:
                for j in i:
                    if(len(j)>2):
                        for m in range(len(symbols)):
                            j = np.char.replace(j, symbols[m], '')
                            j = np.char.replace(j, ",", "")
                            j = np.array2string(j)
                            j = j.replace("'","")
                        if(j not in v):
                            v.append(j)
        else:
            rules = []
            for i in docregs:
                for j in i:
                    rules.append(j)
            vocab0 = mainvocab
        
            for j in vocab0:
                if(len(j)>2):
                    for m in range(len(symbols)):
                        j = np.char.replace(j, symbols[m], '')
                        j = np.char.replace(j, ",", "")
                        j = np.array2string(j)
                        j = j.replace("'","")
                    if(j not in v):
                        v.append(j)
    
    if('All' not in docinput):
        vdef = []
        for i in v:
            if(i in mainvocab and len(i)>2):
                vindex = mainvocab.index(i)
                vdef.append(vocabdef[vindex])
    else:
        vdef = vocabdef
        
    
    return rules, v, vdef, docinput

In [6]:
def querypreprocess(query, qvocab, definitions, finaltopics):
    if('regulations' in qvocab):
        qvocab.remove('regulations')
    if('rules' in qvocab):
        qvocab.remove('rules')
    if('rule' in qvocab):
        qvocab.remove('rule')
    if('chapter' in qvocab):
        qvocab.remove('chapter')
    if('section' in qvocab):
        qvocab.remove('section')
    if('sub' in qvocab):
        qvocab.remove('sub')
    if('SEBI' in qvocab):
        qvocab.remove('SEBI')
    if('means' in qvocab):
        qvocab.remove('means')
    if('shall' in qvocab):
        qvocab.remove('shall')
    if('Securities' in qvocab):
        qvocab.remove('Securities')
    if('Exchange' in qvocab):
        qvocab.remove('Exchange')
    if('pertaining' in qvocab):
        qvocab.remove('pertaining')
    if('India' in qvocab):
        qvocab.remove('India')
        
    query = ""
    for i in qvocab:
        query = query + " " + i
        
    importantwords = []
    for i in qvocab:
        flag = 0
        for j in definitions:
            if(i in j):
                flag = 1
                break
        if(flag == 1):
            importantwords.append(i)

    expansionwords = []
    for i in qvocab:
        flag=0
        for j in finaltopics:
            for k in j:
                if(i == k):
                    flag=1
        if(flag==1):
            if(i not in importantwords):
                importantwords.append(i)
        if(flag==0):
            for j0 in definitions:
                if(i in j0):
                    expansionwords.append(j0)
                    break
    
    qnew = query
    for j in expansionwords:
        k=0
        for i in definitions:
            if(i==j):
                s = i
                s = s + defvalues[k]
                qnew = qnew + ' ' + s
                break
            k=k+1
        else:
            qnew = qnew + ' ' + j
    query = qnew
    
    sent = nlp(query)
    t=0
    for token in sent:
        if(str(token) in importantwords):
            if(t!=0):
                if(str(sent[t-1]) not in importantwords):
                    importantwords.append(str(sent[t-1]))
            if(token.tag_ == 'VB'):
                importantwords.append(str(token))
        t+=1
    
    return query, qvocab, importantwords, expansionwords

## Regulations Retrieval Module

In [7]:
def convert_lower_case(data):
    return np.char.lower(data)

def remove_stop_words(data):
    stop_words = STOPWORDS
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [8]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data)
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = stemming(data)
    return data

In [9]:
def tfidfreg(docregs, docinput, category):
    
    if(category == 'regulations'):
        d = [] 
    
        for i in docinput:
            if(i in docs):
                ind = docs.index(i)
                r0 = docregs[ind]
                d.append(r0)
            
        if(len(d)>0):
            docregs = d

    regstring = []
    origregstring = []
    for i in docregs:
        for j in i:
            origregstring.append(j)
            data = preprocess(j)
            regstring.append(data)        
    wordregs = regstring
    
    t=0
    remind = []
    for i in regstring:
        if(i=='\xa0\n' or i == ''):
            remind.append(t)
            regstring.remove(i)
            wordregs.remove(wordregs[t])
        t+=1
        
    origreg = []
    t=0
    while(t<len(origregstring)):
        if(t not in remind):
            origreg.append(origregstring[t])
        t+=1
            
    wordregs = regstring
    
    DF = {}
    wlist = []
    for i in range(len(wordregs)):
        tokens = regstring[i]
        for w in tokens.split():
            try:
                DF[w].add(i)
            except:
                DF[w] = {i}
                wlist.append(w)
                
    for i in DF:
        DF[i] = len(DF[i])
    
    total_vocab = [x for x in DF]
    total_vocab_size = len(DF)

# TFIDF Calculation here
    tf_idf = {}
    N = len(regstring)
    for i in range(N):
        tokens = regstring[i].split()
        counter = Counter(tokens)
        for token in np.unique(tokens):
            if(token in wlist):
                tf = counter[token]/len(tokens)
                df = DF[token]
                idf = np.log((N+1)/(df+1))
                try:
                    tf_idf[token] = tf_idf[token] + tf*idf
                except:
                    tf_idf[token] = tf*idf                
            
    return DF, N, tf_idf, total_vocab, total_vocab_size, wordregs, regstring, origreg

In [10]:
def q_vector(qvocab, N, total_vocab, DF):
    Q = np.zeros((len(total_vocab)))
    querytokens = qvocab
    counter = Counter(querytokens)
    words_count = len(querytokens)

    query_weights = {}
    
    for token in np.unique(querytokens):
        if(token in DF):
            tf = counter[token]/words_count
            df = DF[token]
            idf = math.log((N+1)/(df+1))

            try:
                ind = total_vocab.index(token)
                Q[ind] = tf*idf
            except:
                pass
    return Q

In [11]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [12]:
def cosine_similarity(k, query, total_vocab, N, qvocab, DF, origreg, D, importantwords):
    d_cosines = []
    
    query_vector = q_vector(qvocab, N, total_vocab, DF)

    t=0
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        t=t+1
        
    out = np.array(d_cosines).argsort()[::-1]
    answers = []
    for i in out:
        flag=0
        for j in qvocab:
            if(j in origreg[i].split()):
                if(j in importantwords):
                    flag=flag+2
                else:
                    flag=flag+1
        if(flag>1):
            answers.append([flag,origreg[i]])

    answers.sort(reverse=True)
    
    if(k<len(answers)):
        return answers[0:k]
    else:
        return answers


In [13]:
def vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg,importantwords):
    D = np.zeros((N, len(total_vocab)))
    for i in tf_idf:
        try:
            ind = total_vocab.index(i[1])
            D[i[0]][ind] = tf_idf[i]
        except:
            pass
    A = cosine_similarity(k, query, total_vocab, N, qvocab, DF, origreg, D, importantwords)

    return A

In [14]:
def answermod(A, queryinput):
    
    ans = []
    ansreg = []
    
    t=0
    while(t<len(A)):
        A[t][0] = 'title'
        A[t][1] = [A[t][1]]
        t+=1
    t=0
    df = pd.DataFrame(A,columns=['title','paragraphs'])
    
    cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib')
    cdqa_pipeline.fit_retriever(df=df)
    
    prediction = cdqa_pipeline.predict(queryinput)
    
    return prediction[0], prediction[2]

In [ ]:
def QnAmodel(queryinput, docinput,k,category):
    print('Preprocessing query......')
    qvocab, query = queryvocab(queryinput)
    if(category == 'regulations'):
        rules, vocab, vdef, docinput = regextract(docinput, docs, docregs, mainvocab, vocabdef, queryinput, glossary)
        query, qvocab, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('tf-idf calculation in progress.....')
        DF, N, tf_idf, total_vocab, total_vocab_size, wordregs, regstring, origreg = tfidfreg(docregs,docinput,category)

#  Can uncomment below lines to save dump
#         with open(DATA + 'tfidf_regs.pkl','wb') as f:
#             pickle.dump(tf_idf,f)


        print('Extracting relevant answer regulations.......')
        A = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords)
        Aold = A
    elif(category == 'misc'):
        
        rules = ctext
        rules.extend(itext)
        
        vocab = miscvocab
        N = 447
        
        with open(DATA + 'misc_tf_idf.pkl', 'rb') as f:
            tf_idf = pickle.load(f)
        with open(DATA + 'misc_total_vocab.pkl', 'rb') as f:
            total_vocab = pickle.load(f)
        with open(DATA + 'misc_DF.json') as f:
            DF = json.load(f)
        with open(DATA + 'misc_origreg.pkl', 'rb') as f:
            origreg = pickle.load(f)
        
        query, qvocab, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('Extracting relevant answer regulations.......')

        A = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords)
        Aold = A
        
        sentences = csent
        sentences.extend(isent)
        Anew = []
        for ans in A:
            ind = rules.index(ans[1])
            for sent in sentences[ind]:
                Anew.append([ans[0],sent])
        A = Anew
            
    elif(category == 'legal case'):
        rules = ltext
        vocab = casevocab
        N = 1496
        
        with open(DATA + 'case_tf_idf.pkl', 'rb') as f:
            tf_idf = pickle.load(f)
        with open(DATA + 'case_total_vocab.pkl', 'rb') as f:
            total_vocab = pickle.load(f)
        with open(DATA + 'case_DF.json') as f:
            DF = json.load(f)
        with open(DATA + 'case_origreg.pkl', 'rb') as f:
            origreg = pickle.load(f)
        
        query, qvocab, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('Extracting relevant answer regulations.......')

        A = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords)
        Aold = A
        
        sentences = lsent
        Anew = []
        for ans in A:
            ind = rules.index(ans[1])
            for sent in sentences[ind]:
                Anew.append([ans[0],sent])
        A = Anew
        
    else:
        return None
        
    print('Extracting answer')
    answer, ansreg = answermod(A, queryinput)
    return answer, ansreg, Aold
#     return Aold

## User input

In [16]:
queryinput = "What are the legal  provisions  on  appointment  and  re-appointment  of directors  of  a listed entity?"
docinput = 'All'
category = 'regulations'
answer, ansreg, Aold = QnAmodel(queryinput,docinput,15, category)

Preprocessing query......
tf-idf calculation in progress.....
Extracting relevant answer regulations.......
Extracting answer


/home/spawnfire/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


In [17]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)


What are the legal  provisions  on  appointment  and  re-appointment  of directors  of  a listed entity?


chapter IV


(2) The listed entity which has listed its specified securities shall comply with the corporate governance provisions as specified in chapter IV which shall be implemented in a manner so as to achieve the objectives of the principles as mentioned below. (a) The rights of shareholders: The listed entity shall seek to protect and facilitate the exercise of the following rights of shareholders: (i)  right to participate in, and to be sufficiently informed of, decisions concerning fundamental corporate changes. (ii) opportunity to participate effectively and vote in general shareholder meetings. (iii)being  informed  of  the  rules,  including  voting  procedures  that  govern  general shareholder meetings. (iv) opportunity  to  ask  questions  to  the  board  of  directors,  to  place  items  on  the agenda of general meetings, and to propose resolutions, subject to reas

In [18]:
queryinput = "As of date, what are the provisions for a person to get appointed of persons who fail to get elected as Whole time Directors /Managing Directors at the general meeting of a listed entity?"              
docinput = 'All'
category = 'misc'
answer, ansreg, Aold = QnAmodel(queryinput,docinput,10, category)

Preprocessing query......
Extracting relevant answer regulations.......
Extracting answer


In [19]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)


As of date, what are the provisions for a person to get appointed of persons who fail to get elected as Whole time Directors /Managing Directors at the general meeting of a listed entity?


the maximum number of public companies in which a person can be appointed as a director shall not exceed ten


twenty companies at the same time: Provided that the maximum number of public companies in which a person can be appointed as a director shall not exceed ten.


In [20]:
queryinput = "What are the key requirements for issuer companies to list under Innovators Growth Platform (IGP)?"
docinput = 'All'
category = 'misc'
answer, ansreg, Aold = QnAmodel(queryinput,docinput,10, category)

Preprocessing query......
Extracting relevant answer regulations.......
Extracting answer


In [21]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)


What are the key requirements for issuer companies to list under Innovators Growth Platform (IGP)?


integrity, experience and solvency requirements


facilitate fund raising by fraudulent entities. It is therefore important to specify integrity, experience and solvency requirements applicable to crowdfunding platform owner and the key persons associated with it. Therefore it is proposed that any online offering or issue or sale through the internet can be made only through a SEBI recognized crowdfunding platform.


In [22]:
for a in Aold:
    print(a[1])
    print("\n======\n")

Discussion Paper on1. Alternate Capital Raising Platform and2. Review of other regulatory requirements1. Background1.1. With a view to provide easier exit options for informed investors like Angel Investors, Venture Capital Funds (VCFs), etc, SEBI Board in its meeting held on June 25, 2013, approved a proposal to permit listing of SMEs in Institutional Trading platform (ITP) without having to make an IPO.1.2. While such companies are listed on the ITP, they are not permitted to raise equity capital through public issues though they can continue to make private placements. SEBI has been in receipt of suggestions to permit capital raising in the said ITP platform.1.3. Further, as part of the regular consultative process, SEBI in the recent past interacted with iSPIRT (which stands for 'Indian Software Product Industry Round Table', a body dedicated to promote existing Indian software product companies). A few Indian start-ups were also present during the said interaction. It was highligh

In [23]:
def get_res(query, docinput, category, qn_ind):
    
    A = QnAmodel(query, docinput, 10, category)

    out_file = "temp_results_annotated_tfidf/" + qn_ind + ".txt"
    out_fptr = open(out_file, 'w+')
    print("Query: ", query, file=out_fptr)
    print("docinput: ", docinput, file=out_fptr)
    print("category: ", category, file=out_fptr)
    
    print("Printing top 10 results from retreiver model:", file=out_fptr)
    for num_id, i in enumerate(A[:10]):
        print("Result ", num_id, ":", file=out_fptr)
        print(i, file=out_fptr)
        print("=====", file=out_fptr)
    out_fptr.close()

In [ ]:
queries_case = ["I am a shareholder in xyz company with my total share capital being 7%. I sold a part of my share percent which reduced my holding by 3%. Will I be required to disclose about the same to under Form C to the company since my total shareholding has fallen below 5%?"
               ,"I am a shareholder in abc company with a shareholding of 10%. I recently acquired a few more shares which has increased my shareholding by 5%, bringing it to a total of 15%. What all disclosures do I need to make?"
               ,"PQR, a corporate entity has purchased 8986 shares and sold 928600 shares of ASQ.  PQR also purchased 9,86,094 shares from off-market. As of 31st December, 2004, PQR was holding 9,86,094 shares which was which was 9.26% of the share capital of ASQ. By March 31st, its shareholding was reduced to 0. What all violations have been made?"
               ,"GHI is a corporate entity that is a promoter of SPZ. As on March 2005, it held 2,30,59,317 shares constituting of 30.87% of the total shares. By the end of September 2005, its shareholding reduced by 1,41,92,187 to 19% . What all violations have been made?"
               ,"TPR is a company that is listed on both BSE & NSE. STC is a promoter of TPR. TPR took out Rights Issue and through that allotted 16,49,653 partly paid-up shares to STC on 2nd October, 2012. Consequently, the total shares of STC  increased from 1,03,18,427 constituting to 4.98% of the share capital to 1,19,68,080 shares constituting 5.06% of the total share capital of TPR. STC was informed about the allotment on 10th October, 2012. However, STC did not make disclosures about its increased shareholding under 13(1) of PIT Regulations and believed that since it was already a promoter of TPR, it instead made disclosure under 14(4A) of PIT Regulations. Accordingly it made disclosure under 13(4A) of PIT Regulations under Form D with a delay of 1 month and not under 13(1) of PIT Regulations under Form A. Is there any violation?"
               ,"Mr. X is a promoter and managing director of GTF Pharmaceuticals. He has pledged certain shares of the company and the pledge on these shares was subsequently released. Mr. X had created a pledge 1,20,000 shares on 11.12.2012 and release of pledge of 5,00,000 shares had taken place on 27.12.2012. Later it was observed that shareholding of Mr. X was changed after some off-market transactions. Mr. X has contended that 1,20,000 shares were pledged with YZ Financial on 11.12.2012 and the said transaction was intimated to stock exchanges on 21.12.2012 and there was a delay of 3 days and the same may be condoned. On 27.12.2012 the pledge on 5 lakh shares were released by DTR and the same was duly intimated to stock exchanges on 04.01.2013. With respect to the off-market transaction, Mr. X has contended that the same was No an off-market transaction but was only a transfer from the his one DP account to its another DP account. It has been alleged that Mr. X did not make the requisite disclosures under Regulation 31(1) &31(2) read with 31(3) of the SAST Regulations, 2011 and Regulation 13(4) & (4A) read with 13(5) SEBI (Prohibition of Insider Trading) Regulations,1992 (PIT Regulations). Has the violation occurred?"
               ,"Mr. X is a promoter and managing director of GTF Pharmaceuticals. He has pledged certain shares of the company and the pledge on these shares was subsequently released. Mr. X had created a pledge 1,20,000 shares on 11.12.2012 and release of pledge of 5,00,000 shares had taken place on 27.12.2012. Later it was observed that shareholding of Mr. X was changed after some off-market transactions. Mr. X has contended that 1,20,000 shares were pledged with YZ Financial on 11.12.2012 and the said transaction was intimated to stock exchanges on 21.12.2012 and there was a delay of 3 days and the same may be condoned. On 27.12.2012 the pledge on 5 lakh shares were released by DTR and the same was duly intimated to stock exchanges on 04.01.2013. With respect to the off-market transaction, Mr. X has contended that the same was No an off-market transaction but was only a transfer from the his one DP account to its another DP account. It has been alleged that Mr. X did not make the requisite disclosures under Regulation 31(1) &31(2) read with 31(3) of the SAST Regulations, 2011 and Regulation 13(4) & (4A) read with 13(5) SEBI (Prohibition of Insider Trading) Regulations,1992 (PIT Regulations). How much penalty should be imposed?"]

In [ ]:
for ind, queryinput in enumerate(queries_case):
    get_res(queryinput, 'All', 'legal case', "CaseFile_"+str(ind))

In [ ]:
queries_new_micro = ["What are some of the activities that being outsourced by intermediaries in the securities market?"
                    ,"Under what conditions can approval from shareholders be dispensed with for reclassification of promoter to public?"
                    ,"What is transmission of shares?"
                    ,"What are the provisions for transfer of shares?"]

In [ ]:
for ind, queryinput in enumerate(queries_new_micro):
    get_res(queryinput, 'All', 'misc', "New_Microsoft_doc_"+str(ind))

In [ ]:
queries_pit = ["Is it insider trading if I know an employee of a company and I buy that company’s stock?"
              ,"How much amount will be provided as reward for informants in insider trading cases by Securities and Exchange Board of India (SEBI)?"
              ,"How do CEOs and their families trade stock from their company without it being insider trading?"
              ,"When do CEOs and their families trade stock from their company without it being insider trading?"
              ,"What should be done by a company when it suspects insider trading by an employee?"
              ,"Who is a compliance officer and what is its role in preventing Insider trading?"]

In [ ]:
for ind, queryinput in enumerate(queries_pit):
    get_res(queryinput, 'All', 'regulations', "PIT_"+str(ind))

In [ ]:
queries_query_testing = ["What if a stock delisted from the market?"
                        ,"What can a person do with shares that are now delisted?"
                        ,"Why companies are delisting their shares?"
                        ,"If a company is about to delist and is ready to buy back my shares and I didn't sell, what will happen?"
                        ,"How does an initial public offering get valued?"
                        ,"How does a company offer initial public offering?"
                        ]

In [ ]:
for ind, queryinput in enumerate(queries_query_testing):
    get_res(queryinput, 'All', 'regulations', "query_testing_"+str(ind))

In [ ]:
queries_sast = ["What are the threshold limits for acquisition of shares / voting rights, beyond which an obligation to   make an open offer is triggered?"
               ,"What is a voluntary open offer?"
               ,"What are the restrictions on acquirers making a voluntary open offer?"]

In [ ]:
for ind, queryinput in enumerate(queries_sast):
    get_res(queryinput, 'All', 'regulations', "sast_"+str(ind))

In [ ]:
queries_sebi_consult = ["What are the legal  provisions  on  appointment  and  re-appointment  of directors  of  a listed entity?"
                       ,"As of date, what are the provisions for a person to get appointed of persons who fail to get elected as Whole time Directors /Managing Directors at the general meeting of a listed entity?"
                       ,"What are the key requirements for issuer companies to list under Innovators Growth Platform (IGP)?"
                       ,"What is meant by 'Migration to Main Board'?"
                       ,"Why are Broad Market Indices created?"
                       ,"Which are the major Domestic & International broad market indices?"]

In [ ]:
for ind, queryinput in enumerate(queries_sebi_consult):
    get_res(queryinput, 'All', 'misc', "sebi_consult_"+str(ind))